In [1]:
from pyswip.prolog import Prolog
from pyswip import registerForeign, Functor, call, Variable
import tempfile
import os
import sys

## Knowledge Base

In [2]:
KB = """

% Used for predicates that are added later
:- dynamic known/3, multivalued/1.

% Rules

% Rule 1:

destination(monument):- distance(close), experience(historical).

% Rule 2

destination(museum) :- distance(close), /+ experience(historical).

% Rule 3

destination(barracks) :- /+ distance(close), experience(historical).

% Rule 4

destination(art_gallery) :- /+ distance(close), /+ experience(historical).

% Rule 5

% destination(cafe) :- distance(close), experience(culinary).


% Askables

distance(X) :- ask(distance, X).

experience(X) :- ask(experience, X).

% experience(X) :- menuask(experience, X, [cultural, historical, culinary])


% System Framework

%% ASK

ask(A, V):-
known(yes, A, V), % succeed if true
!.	% stop looking

ask(A, V):-
known(_, A, V), % fail if false
!, fail.

% If not multivalued, and already known to be something else, don't ask again for a different value.
ask(A, V):-
\+multivalued(A),
known(yes, A, V2),
V \== V2,
!, fail.

ask(A, V):-
read_py(A,V,Y), % get the answer
assertz(known(Y, A, V)), % remember it
Y == "yes".	% succeed or fail

%% MENU ASK
menuask(A, V, Menu):-
known(yes, A, V), % succeed if true
!.	% stop looking

menuask(A, V, Menu):-
known(yes, A, _), % fail if it's not a provided value
!, fail.

menuask(A, V, Menu):-
read_menu_py(A, V, Y, MenuList), % get the answer
asserta(known(yes, A, Y)),
member(Y, Menu),
Y == V.
"""

## Interface Scaffold

In [3]:
prolog = Prolog()

# pull out Prolog built-ins into the Python name-space
retractall = Functor("retractall")
known = Functor("known", 3)
member = Functor('member', 2)

# user input and print functions
def write_py(X):
    print(str(X))
    sys.stdout.flush()
    return True

def read_py(A,V,Y):
    if isinstance(Y, Variable):
        response = input(str(A) + " is " + str(V) + "? ")
        Y.unify(response)
        return True
    else:
        return False

def read_menu_py(A, V, Y, Menu):
    if isinstance(Y, Variable):
        response = input(f"What is the value for {str(A)}? Options: {str(Menu)}")
        Y.unify(response)
    else:
        return False

write_py.arity = 1
read_py.arity = 3
read_menu_py.arity = 4

registerForeign(read_py)
registerForeign(read_menu_py)
registerForeign(write_py)

# Create a temporary file with the KB in it
(FD, name) = tempfile.mkstemp(suffix='.pl', text = "True")
with os.fdopen(FD, "w") as text_file:
    text_file.write(KB)
prolog.consult(name) # open the KB for consulting
os.unlink(name) # Remove the temporary file

call(retractall(known))
destination = [s for s in prolog.query("destination(X).", maxresult=1)]
print("Your recommended destination is " + (destination[0]['X'] + "." if destination else "unknown."))

ERROR: /var/folders/_n/c17wd9k97s7dm583pk_qqyjh0000gn/T/tmp5tt0vi2f.pl:14:43: Syntax error: Operator expected
ERROR: /var/folders/_n/c17wd9k97s7dm583pk_qqyjh0000gn/T/tmp5tt0vi2f.pl:18:28: Syntax error: Operator expected
ERROR: /var/folders/_n/c17wd9k97s7dm583pk_qqyjh0000gn/T/tmp5tt0vi2f.pl:22:31: Syntax error: Operator expected


Your recommended destination is monument.


## Old Rules & Askables

In [7]:
# """
# % Rule 1:** if engine is not turning_over and battery is bad then problem is battery.

# destination(battery):- \+engine(turning_over), battery(bad).

# % Rule 2

# problem(engine_oil_low) :- \+engine(turning_over), warning_light(oil).

# % Rule 3

# battery(bad) :- lights(weak).

# % Rule 4

# battery(bad) :- radio(weak).

# % Rule 5

# problem(engine_flooded):- engine(turning_over), smell(gas).

# % Rule 6 if engine is turning_over and gas_gauge is empty then problem is out_of_gas.

# problem(out_of_gas):- engine(turning_over), gas_gauge(empty).


# % Askables

# engine(X):- ask(engine, X).

# warning_light(X):- ask(warning_light, X).

# lights(X):- ask(lights, X).

# smell(X):- ask(smell, X).

# radio(X):- ask(radio, X).

# gas_gauge(X):- ask(gas_gauge, X).
# """

# Template Example

In [2]:
KB = """
%  Tell prolog that known/3 and multivalued/1 will be added later
:- dynamic known/3, multivalued/1.

% Enter your KB below this line:

problem(battery) :- \+engine(turning_over), battery(bad).
problem(engine_oil_low) :- \+engine(turning_over), warning_light(oil).
battery(bad) :- lights(weak).
battery(bad) :- radio(weak).
problem(out_of_gas) :- engine(turning_over), gas_gauge(empty).
problem(engine_flooded) :- engine(turning_over), smell(gas).
problem(dirty_car) :- smell(pizza).

% The code below implements the prompting to ask the user:

gas_gauge(X) :- ask(gas_gauge, X).
engine(X) :- ask(engine, X).
lights(X) :- ask(lights, X).
radio(X) :- ask(radio, X).
smell(X) :- ask(smell, X).
warning_light(X) :- ask(warning_light,X).

% Asking clauses

ask(A, V):-
known(yes, A, V), % succeed if true
!.	% stop looking

ask(A, V):-
known(_, A, V), % fail if false
!, fail.

% If not multivalued, and already known to be something else, don't ask again for a different value.
ask(A, V):-
\+multivalued(A),
known(yes, A, V2),
V \== V2,
!, fail.

ask(A, V):-
read_py(A,V,Y), % get the answer
assertz(known(Y, A, V)), % remember it
Y == "yes".	% succeed or fail
"""

In [1]:
# Import necessary packages
import tempfile
from pyswip.prolog import Prolog
from pyswip.easy import *

prolog = Prolog() # Global handle to interpreter

retractall = Functor("retractall")
known = Functor("known",3)

# Define foreign functions for getting user input and writing to the screen
def write_py(X):
    print(str(X))
    sys.stdout.flush()
    return True

def read_py(A,V,Y):
    if isinstance(Y, Variable):
        response = input(str(A) + " is " + str(V) + "? ")
        Y.unify(response)
        return True
    else:
        return False

write_py.arity = 1
read_py.arity = 3
registerForeign(read_py)
registerForeign(write_py)

prolog.consult('knowledge-base.pl') # open the KB for consulting

call(retractall(known))
problem = [s for s in prolog.query("problem(X).", maxresult=1)]
print("Your problem is " + (problem[0]['X'] + "." if problem else "unknown."))

Your problem is battery.
